In [ ]:
# 1. Установка библиотек
!pip install -q langchain chromadb sentence-transformers transformers ipywidgets

In [ ]:
# 2. Импорт библиотек
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# 3. Загрузка текстовых файлов
def load_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

upcoming_text = load_file("upcoming.txt")  # ожидаемые новинки
released_text = load_file("released.txt")  # вышедшие игры

In [ ]:
# 4. Разбиение текстов на куски для поиска
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
upcoming_docs = splitter.split_documents([Document(page_content=upcoming_text)])
released_docs = splitter.split_documents([Document(page_content=released_text)])

In [ ]:
# 5. Создание эмбеддингов и баз данных
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_upcoming = Chroma.from_documents(upcoming_docs, embeddings, collection_name="upcoming")
db_released = Chroma.from_documents(released_docs, embeddings, collection_name="released")

In [ ]:
# 6. Подключение генератора текста
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device=-1)

def ask(query, db):
    retriever = db.as_retriever(search_kwargs={"k": 3})
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([d.page_content for d in docs])
    prompt = f"Используя стиль из этого текста:\n{context}\n\nНапиши обзор на: {query}"
    result = generator(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)
    return result[0]["generated_text"]

In [ ]:
# 7. Создание интерфейса с выпадающим списком
dropdown = widgets.Dropdown(
    options=['Новинки', 'Вышедшие игры'],
    value='Новинки',
    description='База:',
)

query_input = widgets.Text(
    value='',
    description='Запрос:',
    placeholder='Название игры...',
    layout=widgets.Layout(width='70%')
)

output_area = widgets.Output()

def on_button_click(b):
    output_area.clear_output()
    db = db_upcoming if dropdown.value == 'Новинки' else db_released
    result = ask(query_input.value, db)
    with output_area:
        print(result)

button = widgets.Button(description="Сгенерировать обзор")
button.on_click(on_button_click)

display(dropdown, query_input, button, output_area)